In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

## Load Model

In [ ]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "pythia-2.8b",
    # "pythia-70m-deduped",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
    fold_ln=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


## Load SAE

In [ ]:


path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

 ** What is a correlative conjunction?  **
 
Correlative conjunctions are conjunctions used to illustrate how two words or phrases within a sentence relate to each other. Correlative conjunctions always come in pairs. 

Though they can illustrate a correlation between the two words or phrases, they don’t necessarily have to.

In [ ]:
# Either - or - Success
prompt = f"Either you are with me,"
answer = " or"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Such - that - Fail
prompt = f"Such is the intensity of the pollen outside,"
answer = " that"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Both - And - Success
prompt = f"My parents went to both Hawaii"
answer = " and"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# No sonner - than - Success
prompt = f"She would no sooner cheat on an exam"
answer = " than"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Rather - than - Pass
prompt = f"They would rather go to the movies"
answer = " than"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Not only - but - Pass
prompt = f"Not only did my boyfriend buy me a Nintendo Switch,"
answer = " but"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# not only - but also - Pass
prompt = f"Not only did my boyfriend buy me a Nintendo Switch, but"
answer = " also"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# as many - as - Pass
prompt = f"There were as many applicants"
answer = " as"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Whether - or - Pass
prompt = f"Whether you bike"
answer = " or"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Neither - nor - Pass
prompt = f"I wasn’t hired at any of the companies I’d applied to. Neither my experience"
answer = " nor"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

In [ ]:
# The more... the more - Pass
prompt = f"the more you learn, the" #more you realize how much you don't know"
answer = " more"
# utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# less on ... and -> more - Pass
prompt = f"I think we should focus less on talking about doing the work and"#more on doing the work"
answer = " more"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)



In [ ]:
# If then - fail
prompt = f"If it is the case that it rains,"
answer = " then"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)

# Unless - Otherwise
prompt = f"Unless we hear"
answer = " otherwise"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)



# Red-Teaming Features

In [ ]:
prompt = "When Bob and Michael went to the shops, Bob gave the shopping to"
answer = " Michael"
# prompt = "All's fair in love and"
# answer = " war"
# prompt = " The cat is cute. The dog is"
# prompt = " Alice, with her keen intelligence and artistic talent, discussed philosophy with Bob, who shared her intellect and also possessed remarkable culinary skills, while"
# answer = " cute"
model.reset_hooks()
utils.test_prompt(prompt, answer, model)

HEAD_HOOK_RESULT_NAME = "blocks.10.attn.hook_z"
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
HEAD_IDX = 7
def hook_to_ablate_head(head_output: Float[Tensor, "batch seq_len head_idx d_head"], hook: HookPoint, head = (LAYER_IDX, HEAD_IDX)):
    print(hook.layer(), hook.name)
    assert head[0] == hook.layer(), f"{head[0]} != {hook.layer()}"
    assert ("result" in hook.name) or ("q" in hook.name) or ("z" in hook.name)
    head_output[:, :, head[1], :] = 0
    return head_output

with model.hooks(fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)]):
    utils.test_prompt(prompt, answer, model)

In [ ]:
import joseph
reload(joseph.analysis)
from joseph.analysis import *


token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt + answer], model, sparse_autoencoder, head_idx_override=7)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence",
               "top_k_features"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm")

In [ ]:
POS_INTEREST = 14# index from 0.
print(token_df.shape)
print(feature_acts.shape)
print(token_df["unique_token"][POS_INTEREST]) 
feature_acts_of_interest = feature_acts[POS_INTEREST]
plot_line_with_top_10_labels(feature_acts_of_interest, "", 30)
vals, inds = torch.topk(feature_acts_of_interest,64)
print(vals.nonzero().shape)
print(inds)

In [ ]:
import webbrowser
from IPython.core.display import display, HTML

path_to_html = "../week_8_jan/gpt2_small_features"
def render_feature_dashboard(feature_id):
    
    path = f"{path_to_html}/data_{feature_id}.html"
    
    
    print(f"Feature {feature_id}")
    
    if os.path.exists(path):
        # with open(path, "r") as f:
        #     html = f.read()
        #     display(HTML(html))
        webbrowser.open_new_tab("file://" + os.path.abspath(path))
    else:
        print("No HTML file found")
    
    return

for feature in inds:
    render_feature_dashboard(feature)

To do:
- Check whether we can use only the top_k features and get the same result.
- Quickly estimate the counter-factual for why features fired in terms of tokens / this sentence.
- Use this to reduce the dimensionality of what we're thinking about?
- Select a feature, red-team it as hard as possible (as automatically as possible)

## Top - K forward Pass

In [ ]:
tokens = model.to_tokens([prompt + answer])
layer_idx = sparse_autoencoder.cfg.hook_point_layer
head_idx = 7
position_of_interest = 14

original_act = original_cache[sparse_autoencoder.cfg.hook_point]
# token_df["q_norm"] = torch.norm(original_act, dim=-1)[:,1:].flatten().tolist()
sae_out, feature_acts, _, mse_loss, _ = sparse_autoencoder(original_act)
head_hook_query_name = utils.get_act_name("q", layer_idx)
head_hook_resid_name = utils.get_act_name("resid_pre", layer_idx)

@torch.no_grad()
def get_top_k_sae_approximation(sparse_autoencoder, feature_acts, top_k_features = None, top_k = 10):
    
    # here we're using the topk by activation.
    # this is the default. 
    if top_k_features is None:
        top_k_features = torch.topk(feature_acts, top_k, dim=2, sorted=True).indices[0]
    
    feature_acts_top_k = torch.zeros_like(feature_acts)
    feature_acts_top_k[:, :, top_k_features] = feature_acts[:, :, top_k_features]
    new_sae_out = (feature_acts_top_k @ sparse_autoencoder.W_dec) + sparse_autoencoder.b_dec
    return new_sae_out

tok_k_tokens = 10
max_features = (feature_acts[:, position_of_interest, :] > 0).sum().item()

logits = model(tokens, return_type="logits")
top_k_vals, topk_token_inds = torch.topk(logits[0,-2,:], tok_k_tokens)
topk_token_strs = [model.tokenizer.decode(i) for i in topk_token_inds]
top_k_feature_vals, top_k_feature_inds = torch.topk(feature_acts[:, position_of_interest,:], max_features, dim=1, sorted=False)


mary_unembed_dot_product = (sparse_autoencoder.W_dec[top_k_feature_inds.squeeze()] @ model.W_U[:,model.tokenizer.encode(" Mary")]).detach().cpu()

# torch.topk(mary_unembed_dot_product, 10).indices

assert  (logits[0,-2,topk_token_inds] == top_k_vals).all().item()


vals = []
# get the top k features by activation, and construct a new sae out 
k_s = range(max_features)
for top_k in tqdm(k_s):
    
    
    # Mary Unembed Ranking
    # top_k_features   =top_k_feature_inds[:,torch.topk(mary_unembed_dot_product, top_k, dim=0).indices].squeeze()
    # new_sae_out = get_top_k_sae_approximation(sparse_autoencoder, feature_acts,  top_k_features=top_k_features)
    
    # Feature activation ranking
    new_sae_out = get_top_k_sae_approximation(sparse_autoencoder, feature_acts, top_k = top_k)
    def replacement_hook(resid_pre, hook, new_resid_pre=new_sae_out, position = position_of_interest):
        resid_pre[:,position,:] = new_resid_pre[:,position,:]
        return resid_pre
    
    with model.hooks(fwd_hooks=[(head_hook_resid_name, replacement_hook)]):
        logits = model(tokens, return_type="logits")
        token_logits = logits[0,position_of_interest,topk_token_inds]
        
        vals.append(token_logits.tolist())
        
        
tmp = pd.DataFrame(
    vals,
    index = [f"feature_{i}" for i in top_k_feature_inds.flatten().tolist()], 
    columns = topk_token_strs).T

px.line(tmp.T, title="Top k features vs. token logits")

## Features over tokens

In [ ]:
features_acts_by_token_df = pd.DataFrame(
    feature_acts[0,:,top_k_feature_inds[0,:]].detach().cpu().T,
    index = [f"feature_{i}" for i in top_k_feature_inds.flatten().tolist()],
    columns = token_df["unique_token"])

px.line(features_acts_by_token_df.sort_values(" to/14", ascending=False).T, title="Top k features by activation")

## Feature Clustering

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.01,
    n_components=2,
    metric="cosine",
    random_state=42,
)


ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec[top_k_feature_inds[0].flatten()].detach().cpu().numpy())

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=3,
    metric="euclidean",
)

clusterer.fit(ummap_result)


temp_df = pd.DataFrame(
    {"feature": top_k_feature_inds.flatten().detach().cpu(), 
        "ummap_x": ummap_result[:,0],
        "ummap_y": ummap_result[:,1],
        "cluster": [f"cluster_{i}" for i in clusterer.labels_],
        })


fig = px.scatter(
        temp_df,#[~temp_df.outlier],
        x="ummap_x",
        y="ummap_y",
        color="cluster",
        # color_continuous_midpoint=0,
        color_continuous_scale="RdBu", 
        hover_name="feature",
        opacity=0.5,
        template="plotly",
    )

# make it wide and tall
fig.update_layout(height=400, width=600)
    
    
fig.show()

## DLA

In [ ]:
decomp, labels = original_cache.get_full_resid_decomposition(layer =  10, expand_neurons=False, return_labels=True)
print(decomp.shape)
inds = top_k_feature_inds.squeeze()
tok1 = " Michael"
tok2 = " Bob"
test = (decomp[:,0,position_of_interest] @ model.W_U[:,model.tokenizer.encode(tok1)]).detach().cpu()

tmp = pd.DataFrame(test.detach().cpu().numpy().T, columns = labels, index = [tok1])
px.line(
    tmp.T
).show()



## DFA

In [ ]:
# now let's do DLA
decomp, labels = original_cache.get_full_resid_decomposition(layer =  10, expand_neurons=False, return_labels=True)
print(decomp.shape)
inds = top_k_feature_inds.squeeze()
test = (decomp[:,0,position_of_interest] @ sparse_autoencoder.W_enc[:,inds])
# test = (decomp[:,0,-1] @ sparse_autoencoder.W_dec[inds].T) / sparse_autoencoder.W_enc[:,inds].norm(dim=0)
tmp = pd.DataFrame(test.detach().cpu().numpy().T, columns = labels, index = [f"feature_{i}" for i in inds])
px.line(
    tmp.T[tmp.T.index.str.contains("mlp")]
).show()

px.line(
    tmp.T[tmp.T.index.str.contains("L")]
).show()

# test = (decomp[:,0,-1] @ sparse_autoencoder.W_enc[:,inds])
test = (decomp[:,0,position_of_interest] @ sparse_autoencoder.W_dec[inds].T) / sparse_autoencoder.W_enc[:,inds].norm(dim=0)
tmp = pd.DataFrame(test.detach().cpu().numpy().T, columns = labels, index = [f"feature_{i}" for i in inds])
px.line(
    tmp.T[tmp.T.index.str.contains("mlp")]
).show()
px.line(
    tmp.T[tmp.T.index.str.contains("L")]
).show()

In [ ]:
import hdbscan

log_feature_sparsity = torch.load("../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/log_feature_sparsity_5000_4.pt")


def get_sae_df(sparse_autoencoder, min_cluster_size=10):
    
    W_enc_normalized = sparse_autoencoder.W_enc.cpu() / sparse_autoencoder.W_enc.cpu().norm(dim=-2, keepdim=True)
    d_e_projection = (sparse_autoencoder.W_dec.cpu() @ sparse_autoencoder.W_enc.cpu()).diag().detach().cpu()
    # px.histogram(d_e_projection, nbins=100, log_x=False, title="d_e projection").show()


    d_e_projection_normalized = (sparse_autoencoder.W_dec.cpu() @ W_enc_normalized.cpu()).diag().detach().cpu()
    # px.histogram(d_e_projection_normalized, nbins=100, log_x=False, title="d_e projection").show()
    
    temp_df = pd.DataFrame({
    "log_feature_sparsity": log_feature_sparsity,
    "d_e_projection": d_e_projection,
    "d_e_projection_normalized": d_e_projection_normalized,
    "b_enc": sparse_autoencoder.b_enc.detach().cpu(),
    "feature": [f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)],
    "index": torch.arange(sparse_autoencoder.cfg.d_sae)
    })

    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    clusterer.fit(temp_df[["log_feature_sparsity", "d_e_projection", "b_enc"]].values)
    temp_df["cluster"] = clusterer.labels_
    temp_df["cluster_categorical"] = temp_df["cluster"].astype(str)
    temp_df["outlier"] = (temp_df.log_feature_sparsity < -9) | (temp_df.cluster == -1) | (temp_df.d_e_projection < -10000)
    
    return temp_df

sae_df = get_sae_df(sparse_autoencoder, 50)
display(sae_df.head())

# px.bar(sae_df.cluster.value_counts().reset_index(), y = "count", color = "cluster", title = "Cluster sizes").show()


def get_anthropic_3d_scatter(temp_df, remove_outliers_and_cluster = True):

    if remove_outliers_and_cluster:
        fig = px.scatter_3d(
            # temp_df,#[~temp_df.outlier],
            temp_df[~temp_df.outlier],
            x="log_feature_sparsity",
            y="d_e_projection",
            z="b_enc",
            color="cluster",
            # color="index",
            hover_name="feature",
        )
        
    else:
        fig = px.scatter_3d(
            temp_df,
            x="log_feature_sparsity",
            y="d_e_projection",
            z="b_enc",
            color="cluster",
            # color="index",
            hover_name="feature",
        )
    fig.update_layout(
        scene = dict(
            xaxis_title='log10 feature sparsity',
            yaxis_title='D/E Projection',
            zaxis_title='Encoder Bias',
        )
    )

    fig.update_traces(marker_size = 2)
    fig.update_layout(
        title="Feature Sparsity vs D/E projection vs Encoder Bias (Excluding Outliers)",
        width=1400,
        height=1300,
    ) 

    # increase font size everywhere
    fig.update_layout(
        font=dict(
            size=18,
        )
    )

    # increase marker size in legend (only)
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.update_scenes(aspectmode='cube')

    return fig

# fig = get_anthropic_3d_scatter(sae_df, remove_outliers_and_cluster=True)

# fig.show()

In [ ]:
top_k_feature_inds.shape

In [ ]:
sae_df.iloc[top_k_feature_inds[0].flatten().tolist()].sort_values("log_feature_sparsity").style.background_gradient(
    subset=["d_e_projection_normalized", "d_e_projection", "log_feature_sparsity", "b_enc"],
    cmap="coolwarm")


# Exploring NMONLI dataset

In [ ]:
import json 



def load_monli_data(path):
    monli_train = []

    with open(path, "r") as f:
        for line in f:
            monli_train.append(json.loads(line))
            
    monli_train = pd.DataFrame(monli_train)
    monli_train["answer"] = monli_train.depth.apply(lambda x: " Yes" if x != -1 else " No").astype(str)
    return monli_train

path = "/Users/josephbloom/GithubRepositories/mats_sae_training/research/week_15th_jan/nmonli_train.jsonl"
monli_train = load_monli_data(path)
path = "/Users/josephbloom/GithubRepositories/mats_sae_training/research/week_15th_jan/nmonli_new.jsonl"
monli_new = load_monli_data(path)

In [ ]:
PREPROMPT = """
Statement: There is not a single person walking in the city.
Hypothesis: There is not a single photographer walking in the city.
We can infer with certainty that the hypothesis is true. Yes or No?
Yes
Statement: The people are not playing clarinets.
Hypothesis: The people are not playing instruments.
We can infer with certainty that the hypothesis is true. Yes or No?
No
Statement: The men were outside trying to keep their voices down so as not to waken any woman indoors
Hypothesis: The men were outside trying to keep their voices down so as not to waken any wife indoors
We can infer with certainty that the hypothesis is true. Yes or No?
No
"""
no_token_id = model.tokenizer.encode("No")[0]
yes_token_id = model.tokenizer.encode("Yes")[0]

for i, row in tqdm(monli_train.sample(3).iterrows(), total=3):
    # print(row["sentence1"])
    # print(row["sentence2"])
    print("----")
    prompt = f"Statement: {row['sentence1']}\nHypothesis: {row['sentence2']}\nWe can infer with certainty that the hypothesis is true. Yes or No?\n"
    print(prompt)
    prompt = f"{PREPROMPT}{prompt}"
    answer = row["answer"]
    print("Answer: ", answer)
    # utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)
    logits = model(prompt, return_type="logits")[:,-1]
    probs = torch.softmax(logits, dim=-1)
    logits_yes = logits[:, yes_token_id].item()
    logits_no = logits[:, no_token_id].item()
    probs_yes = probs[:, yes_token_id].item()
    probs_no = probs[:, no_token_id].item()
    
    print(f"probs_yes: {probs_yes:.2f}, probs_no: {probs_no:.2f}, logits_yes: {logits_yes:.2f}, logits_no: {logits_no:.2f}")


In [ ]:

def nmonli_eval(monli_train, model, n_shots = 3):

    no_token_id = model.tokenizer.encode(" No")[0]
    yes_token_id = model.tokenizer.encode(" Yes")[0]

    probs_yes_list = []
    probs_no_list = []
    logits_yes_list = []
    logits_no_list = []
    ce_loss_list = []
    
    # get premprompt
    preprompt = []
    for i, row in tqdm(monli_train.sample(n_shots).iterrows(), total=len(monli_train)):
        prompt = f"The statement \"{row['sentence1']}\", implies that \"{row['sentence2']}\". Yes or No?{row['answer']}"
        preprompt.append(prompt)
    
    preprompt = "\n".join(preprompt)
    print("Selected Preprompt:\n{}".format(preprompt))
    print("------")

    for i, row in tqdm(monli_train.iterrows(), total=len(monli_train)):
        # print(row["sentence1"])
        # print(row["sentence2"])
        # print("----")
        prompt = f"The statement \"{row['sentence1']}\", implies that \"{row['sentence2']}\". Yes or No?"
        # print(prompt)
        prompt = f"{preprompt}\n{prompt}"
        # prompt = f"{prompt}"
        # print(prompt)
        answer = row["answer"]
        # print("Answer: ", answer)
        # utils.test_prompt(prompt, answer, model, prepend_space_to_answer=False, top_k=3)
        logits = model(prompt, return_type="logits")[:,-1]
        probs = torch.softmax(logits, dim=-1)
        logits_yes = logits[:, yes_token_id].item()
        logits_no = logits[:, no_token_id].item()
        probs_yes = probs[:, yes_token_id].item()
        probs_no = probs[:, no_token_id].item()
        
        target = torch.zeros_like(logits)
        target_token_id = yes_token_id if answer == " Yes" else no_token_id
        target[:, target_token_id] = 1
        ce_loss = torch.nn.CrossEntropyLoss()(logits.unsqueeze(0), target.unsqueeze(0)).item()
        
        probs_yes_list.append(probs_yes)
        probs_no_list.append(probs_no)
        logits_yes_list.append(logits_yes)
        logits_no_list.append(logits_no)
        ce_loss_list.append(ce_loss)
        
    monli_train["probs_yes"] = probs_yes_list
    monli_train["probs_no"] = probs_no_list
    monli_train["logits_yes"] = logits_yes_list
    monli_train["logits_no"] = logits_no_list
    monli_train["ce_loss"] = ce_loss_list
    monli_train["yes_minus_no"] = monli_train["logits_yes"] - monli_train["logits_no"]
    monli_train.head()

    return monli_train

results_df = nmonli_eval(monli_train, model)
px.strip(results_df, orientation="v", y="probs_yes", color="answer", title="Probability of Yes", hover_data=["sentence1", "sentence2"])

In [ ]:
px.strip(results_df, orientation="v", y="probs_yes", color="answer", title="Probability of Yes", hover_data=["sentence1", "sentence2"], height=800, width=800)

In [ ]:
px.histogram(results_df, x="yes_minus_no", color="answer", title="Yes - No", hover_data=["sentence1", "sentence2"], height=800, width=800)

In [ ]:
px.strip(monli_train, x="probs_yes", color="answer", title="Probability of Yes", hover_data=["sentence1", "sentence2"])

In [ ]:
import nltk

nltk.download("words")

from nltk.corpus import words


In [ ]:
d_vocab = model.tokenizer.vocab_size
vocab_df = pd.DataFrame(
    {
        "token": np.arange(d_vocab),
        "string": model.to_str_tokens(np.arange(d_vocab)),
    }
)
vocab_df["string_lower_stripped"] = vocab_df.string.str.lower().str.strip()
vocab_df["is_alpha"] = vocab_df.string.str.match(r"^( ?)[a-z]+$")
vocab_df["is_word"] = vocab_df.string_lower_stripped.isin(words.words())
vocab_df["is_fragment"] = vocab_df.string.str.match(r"^[a-z]+$")
vocab_df["has_space"] = vocab_df.string.str.match(r"^ [A-Za-z]+$")
vocab_df["num_chars"] = vocab_df.string.apply(lambda n: len(n.strip()))
vocab_df

In [ ]:
word_df = vocab_df[vocab_df.is_word & vocab_df.has_space & (vocab_df.num_chars == 5)]
word_df.head(30)

In [ ]:
def make_single_prompt():
    words = [word_df.string.sample().item().strip() for _ in range(3)]
    print(words)
    sorted_words = sorted(words)
    return f"Question: Put the following words in order: {', '.join(words)}\nAnswer: {', '.join(sorted_words)}"
make_single_prompt()

In [ ]:
import circuitsvis as cv

example_prompt = make_single_prompt()
logits, cache = model.run_with_cache(example_prompt)
display(cv.logits.token_log_probs(model.to_tokens(example_prompt), model(example_prompt)[0].log_softmax(dim=-1), model.to_string))

In [ ]:
def make_single_prompt():
    word = short_chars_vocab_df.string.sample().item().strip()
    return f" {word}:" + "".join([f" {c.upper()}" for c in word.strip()])


def make_kshot_prompt(k=3):
    return "\n".join([make_single_prompt() for _ in range(k)])


def make_kshot_prompts(n=10, k=3):
    return [make_kshot_prompt(k) for _ in range(n)]


def get_answer_index(prompts):
    batch_size = len(prompts)
    answer_index = torch.zeros((batch_size, 5), device="cuda", dtype=torch.int64) - 1
    for i in range(batch_size):
        for j in range(5):
            answer_index[i, j] = alphabet.index(prompts[i][2 * j - 9].lower())
    return answer_index


# Variable Binding

In [ ]:
E_0 = "Jack"
A_0 = "London"
E_1 = "Mary"
A_1 = "Paris"
qn_subject = "Jack"
prompt = f"""Answer the question based on the context below. Keep the answer short.
Context: {E_0} lives in the capital city of {A_0}.
{E_1} lives in the capital city of {A_1}.
Question: Which city does {qn_subject} live in?
Answer: {qn_subject} lives in the city of"""
print(prompt)

answer = A_0
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

In [ ]:
import random
# capital_cities = [
#     "Washington", "Ottawa", "Mexico City", "London", "Paris", "Berlin", 
#     "Moscow", "Beijing", "Tokyo", "Canberra", "New Delhi", "Brasília", 
#     "Cairo", "Pretoria", "Rome", "Madrid", "Ankara", "Buenos Aires", 
#     "Riyadh", "Ottawa", "Stockholm", "Helsinki", "Oslo", "Copenhagen", 
#     "Wellington", "Seoul", "Bangkok", "Kuala Lumpur", "Jakarta", "Hanoi"
# ]
american_cities = [
    "Washington", "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose", "Austin",
    "Jacksonville", "Fort Worth", "Columbus", "Charlotte", "San Francisco",
    "Indianapolis", "Seattle", "Denver", "Washington", "Boston", "El Paso",
    "Nashville", "Detroit", "Oklahoma City", "Portland", "Las Vegas", "Memphis",
    "Louisville", "Baltimore", "Milwaukee", "Albuquerque", "Tucson", "Fresno",
    "Mesa", "Sacramento", "Atlanta", "Kansas City", "Colorado Springs",
    "Omaha", "Raleigh", "Miami", "Long Beach", "Virginia Beach", "Oakland",
    "Minneapolis", "Tulsa", "Tampa", "Arlington", "New Orleans", "Wichita",
    "Cleveland", "Bakersfield", "Aurora", "Anaheim", "Honolulu", "Santa Ana",
    "Riverside", "Corpus Christi", "Lexington", "Stockton", "St. Louis",
    "Saint Paul", "Henderson", "Pittsburgh", "Cincinnati", "Anchorage",
    "Greensboro", "Plano", "Newark", "Lincoln", "Orlando", "Irvine",
    "Toledo", "Jersey City", "Chula Vista", "Durham", "Fort Wayne",
    "St. Petersburg", "Laredo", "Buffalo", "Madison", "Lubbock", "Chandler",
    "Scottsdale", "Reno", "Glendale", "Gilbert", "Winston–Salem", "North Las Vegas",
    "Norfolk", "Chesapeake", "Garland", "Irving", "Hialeah", "Fremont",
    "Boise", "Richmond", "Baton Rouge", "Spokane", "Des Moines", "Tacoma",
    "San Bernardino", "Modesto", "Fontana", "Santa Clarita", "Birmingham",
    "Oxnard", "Fayetteville", "Moreno Valley", "Rochester", "Glendale",
]
    

american_names = [
    "Emma", "Noah", "Olivia", "Liam", "Ava", "Ethan", "Sophia", "Mason", 
    "Isabella", "Logan", "Mia", "James", "Charlotte", "Benjamin", "Amelia", 
    "Jacob", "Harper", "Michael", "Evelyn", "Alexander", "Abigail", 
    "William", "Emily", "Jack", "Elizabeth", "Henry", "Madison", 
    "Aiden", "Chloe", "Matthew"
]

names = [
    "Ana", "Lei", "Ivan", "Zoe", "Juan", "Eva", "Liam", "Raj", "Sara", 
    "Ken", "Maya", "Noah", "Aya", "Omar", "Lena", "Yuki", "Chen", 
    "Eli", "Ria", "Hugo", "Tara", "Kai", "Nina", "Leo", "Gita", 
    "Max", "Lila", "Sam", "Zara", "Tom"
]


def generate_all_possible_variable_binding_promts(names, capital_cities, n_prompts=100):
    prompts = []
    A_0s = []
    A_1s = []
    E_0s = []
    E_1s = []
    qn_subjects = []
    answers = []
    decoys = []
    
    for _ in range(n_prompts):
        A_0 = random.choice(capital_cities)
        A_1 = random.choice(capital_cities)
        E_0 = random.choice(names)
        E_1 = random.choice(names)
        qn_subject = E_0 if random.random() < 0.5 else E_1
        answer = A_0 if qn_subject == E_0 else A_1
        decoy = A_1 if qn_subject == E_0 else A_0
        
        prompt = f"""Answer the question based on the context below. Keep the answer short.
Context: {E_0} lives in the capital city of {A_0}.
{E_1} lives in the capital city of {A_1}.
Question: Which city does {qn_subject} live in?
Answer: {qn_subject} lives in the city of"""

        A_0s.append(A_0)
        A_1s.append(A_1)
        E_0s.append(E_0)
        E_1s.append(E_1)
        qn_subjects.append(qn_subject)
        prompts.append(prompt)
        answers.append(answer)
        decoys.append(decoy)
        
    df = pd.DataFrame({
        "A_0": A_0s,
        "A_1": A_1s,
        "E_0": E_0s,
        "E_1": E_1s,
        "qn_subject": qn_subjects,
        "answer": answers,
        "decoy": decoys,
        "prompt": prompts,
    })
        
    return df

df = generate_all_possible_variable_binding_promts(american_names, american_cities, n_prompts=100)
df.head()

df.iloc[0].T

In [ ]:

results_list = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    # print(row["prompt"])
    # print(row["qn_subject"])
    # utils.test_prompt(row["prompt"], row["answer"], model, prepend_space_to_answer=True, top_k=3)
    # print("-----")
    
    logits = model(row["prompt"], return_type="logits")[:,-1]
    probs = torch.softmax(logits.squeeze(), dim=-1)
    
    target_token_id = model.tokenizer.encode(" " + row["answer"].strip())[0]
    decoy_token_id = model.tokenizer.encode(" " + row["decoy"].strip())[0]
    top_unembed_token = model.tokenizer.decode(torch.topk(logits, 1).indices.item())
    logits_target = logits[:, target_token_id].item()
    probs_target = probs[target_token_id].item()
    logits_decoy = logits[:, decoy_token_id].item()
    probs_decoy = probs[decoy_token_id].item()
    
    target = torch.zeros_like(logits)
    target[:, target_token_id] = 1
    ce_loss = torch.nn.CrossEntropyLoss()(logits.unsqueeze(0), target.unsqueeze(0)).item()

    
    # print(f"probs_target: {probs_target:.2f}, probs_decoy: {probs_decoy:.2f}, logits_target: {logits_target:.2f}, logits_decoy: {logits_decoy:.2f}")

    
    results_list.append({
        "E_0": row["E_0"],
        "A_0": row["A_0"],
        "E_1": row["E_1"],
        "A_1": row["A_1"],
        "qn_subject": row["qn_subject"],
        "answer": row["answer"],
        "prompt": row["prompt"],
        "top_unembed_token": top_unembed_token,
        "target_token_id_string": model.tokenizer.decode(target_token_id),
        "decoy_token_id_string": model.tokenizer.decode(decoy_token_id),
        "probs_target": probs_target,
        "probs_decoy": probs_decoy,
        "logits_target": logits_target,
        "logits_decoy": logits_decoy,
    })
    
results_df = pd.DataFrame(results_list)
results_df["logit_diff"] = results_df["logits_target"] - results_df["logits_decoy"]
# results_df
fraction_correct = (results_df.logit_diff > 0).mean()
print(f"Fraction correct: {fraction_correct:.2f}")

In [ ]:
px.strip(
    results_df,
    y= "logit_diff",
    # barmode="group",
    hover_data=["prompt", "qn_subject", "answer", "top_unembed_token", "target_token_id_string", "decoy_token_id_string"],
)

# Task Vectors

In [ ]:
prompt = f"""Up: Down, Top: Bottom, Left:"""
answer = " Right"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

prompt = f"""Happy: Sad, Good: Bad, Hot: Cold, Up:"""
answer = " Down"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

prompt = f"""Fire: Ice, Hot: Cold, Up:"""
answer = " Down"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

In [ ]:
prompt = f"""Up: U, Top: T, Left: L, Right:"""
answer = " R"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

prompt = f"""Cat: C, Dog: D, Mouse: M, Elephant:"""
answer = " E"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

In [ ]:
prompt = f""" A B C D E F G H I J K L M N O P Q R S T U V W X Y"""
answer = " Z"
utils.test_prompt(prompt, answer, model, prepend_space_to_answer=True, top_k=3)

In [ ]:
prompt = "English: I love you. French:"
utils.test_prompt(prompt, " Je", model, prepend_space_to_answer=True, top_k=3)